In [1]:
##  LIBRERIAS  
import os
import numpy as np
import pandas as pd
import re
from unidecode import unidecode
from tqdm.notebook import trange, tqdm
import time
import math

import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString, Polygon

from mpl_toolkits import mplot3d
%matplotlib inline

In [5]:
## LECTURA ... Df Parametros
par_file_name="CLAG1.par"

#print(open(par_file_name, "r").read())
open(par_file_name, "r").close()
l=[re.split("[']", x) for x in open(par_file_name, "r")]
#print(x)
open(par_file_name, "r").close()            #cerrar archivo 
## diccionario de parámetros
param={}
param_l=[]
paran_v=[]

## OJO!! 
# Separador no es el mismo en todos los SO -->  Usar OS

## header:          PARÁMETRO 1
param[1]=l[0][0].strip()
param_l.append(l[0][1].strip().replace(os.sep,""))
## input file:          PARÁMETRO 2
param[2]=l[1][0].strip().rsplit(os.sep, 1)[-1]
param_l.append(l[1][1].strip().replace(os.sep,""))
## columns for data:          PARÁMETRO 3
param[3]=[int(x) for x in l[2][0].strip().split(",")]
param_l.append(l[2][1].strip().replace(os.sep,"")[0:16])
## initial search volume:          PARÁMETRO 4
param[4]=[float(x) for x in l[3][0].strip().split(",")]
param_l.append(l[3][1].strip().replace(os.sep,""))
## Number of Risk Thresholds:          PARÁMETRO 5
param[5]=[int(x) for x in l[4][0].strip().split(",")]
param_l.append(l[4][1].strip().replace(os.sep,"")[0:37])
## Selected risk thresholds:          PARÁMETRO 6
param[6]=[float(x) for x in l[5][0].strip().split(",")]
param_l.append(l[5][1].strip().replace(os.sep,""))
## Number of data in local volume:          PARÁMETRO 7
param[7]=[int(x) for x in l[6][0].strip().split(",")]
param_l.append(l[6][1].strip().replace(os.sep,""))
## Number of units:          PARÁMETRO 8
param[8]=[int(x) for x in l[7][0].strip().split(",")]
param_l.append(l[7][1].strip().replace(os.sep,""))
## Unit names:          PARÁMETRO 9
param[9]=[int(x) for x in l[8][0].strip().split(",")]
param_l.append(l[8][1].strip().replace(os.sep,""))
## not consider values less than:          PARÁMETRO 10
param[10]=[int(x) for x in l[9][0].strip().split(",")]
param_l.append(l[9][1].strip().replace(os.sep,""))
## output files name:          PARÁMETRO 11
param[11]=l[10][0].strip().rsplit(os.sep, 1)[-1]
param_l.append(l[10][1].strip().replace(os.sep,""))
## number of zones:          PARÁMETRO 12
param[12]=[int(x) for x in l[11][0].strip().split(",")]
param_l.append(l[11][1].strip().replace(os.sep,""))
## zone identifiers:          PARÁMETRO 13
param[13]=[int(x) for x in l[12][0].strip().split(",")]
param_l.append(l[12][1].strip().replace(os.sep,""))
## azimuth of search orientation per zone :          PARÁMETRO 14
param[14]=[int(x) for x in l[13][0].strip().split(",")]
param_l.append(l[13][1].strip().replace(os.sep,""))
## Anisotropy ratios:          PARÁMETRO 15
param[15]=[float(x) for x in l[14][0].strip().split(",")]
param_l.append(l[14][1].strip().replace(os.sep,""))
## Exclude data from the same drillhole:          PARÁMETRO 16
param[16]=[int(x) for x in l[15][0].strip().split(",")]
param_l.append(l[15][1].strip().replace(os.sep,""))
## Option to weight data by search expansion:          PARÁMETRO 17
param[17]=[int(x) for x in l[16][0].strip().split(",")]
param_l.append(l[16][1].strip().replace(os.sep,""))

df_parametros = pd.DataFrame(list(zip(list(range(1,18)), param_l, list(param.values()))),
                            columns=['Orden','Descripcion', 'Valor']
                        )


df_parametros






,Orden,Descripcion,Valor
0,1,header,Capping Test Copper Using BHID
1,2,input file,CLAG_MR2022_2.csv
2,3,columns for data,"[1, 2, 3, 4, 6, 5, 7, 8]"
3,4,initial search volume,"[30.0, 30.0, 30.0]"
4,5,Number of Risk Thresholds to consider,[6]
5,6,selected risk thresholds,"[0.1, 0.125, 0.15, 0.175, 0.2, 0.225]"
6,7,number of data in local volume,[21]
7,8,number of units,[2]
8,9,unit names,"[31, 32]"
9,10,do not consider values less than,[0]


In [6]:
##   Lectura CSV compositos .. validar si hay nulos o no

df_CLAG_MR_2022 = pd.read_csv(param[2])
df_CLAG_MR_2022.head(2)
## Requerimiento: No NaN ni missing values
'''
Algoritmo localcapping no admite valores nulos ni missings
función missing_val(df) identifica casos missing y nulos
'''

def missing_val(df):
    cero_val = (df == 0.00).astype(int).sum(axis=0)
    mis_val = df.isnull().sum()
    mz_table = pd.concat([cero_val, mis_val], axis=1)
    mz_table = mz_table.rename(
               columns = {0 : 'Valores  Cero', 1 : 'Valores Missing'})
    mz_table['Total Cero + Missing'] = mz_table['Valores  Cero'] + mz_table['Valores Missing']
    mz_table = mz_table[mz_table.iloc[:,1] != 0].round(1)
    print ("Df Columnas: " + str(df.shape[1]) + " Df Filas: " + str(df.shape[0]) + " \n"      
            "Columnas con Ceros / Missing: " + str(mz_table.shape[0]))
    return mz_table

## 
missing_val(df_CLAG_MR_2022)  




Df Columnas: 8 Df Filas: 4255 
Columnas con Ceros / Missing: 0


,Valores Cero,Valores Missing,Total Cero + Missing


In [7]:
df_CLAG_MR_2022.head(2)

,COUNT,mid_x,mid_y,mid_z,Ag_ppm_BESTEL,NBKRT,AG_SUBZONAS,HOLEID
0,2336,4072.2950,4850.2450,4199.8669,25.146513,32,42,A019
1,2337,4073.5668,4848.7522,4196.7885,40.000000,31,42,A019


In [8]:
# ACONDICIONAR DATAFRAME


'''
Este ordenamiento no limita la seleccion decolumnas por nombre
Hace seleccion en funcion del orden inicial de columnas
'''
df_CLAG_MR_2022_R=df_CLAG_MR_2022[[df_CLAG_MR_2022.iloc[:,0].name,
                                   df_CLAG_MR_2022.iloc[:,1].name,
                                   df_CLAG_MR_2022.iloc[:,2].name,
                                   df_CLAG_MR_2022.iloc[:,3].name,
                                   df_CLAG_MR_2022.iloc[:,4].name,
                                   df_CLAG_MR_2022.iloc[:,5].name,
                                   df_CLAG_MR_2022.iloc[:,6].name,
                                   df_CLAG_MR_2022.iloc[:,7].name]]

df_CLAG_MR_2022=df_CLAG_MR_2022_R

df_CLAG_MR_2022.head(2)


##  $$$  INICILIZAR VALORES PARA 6 UMBRALES
##  y    #ID,BHID,X,Y,Z,CODE,ZONE,VALUE,CAPVAL1,CAPPED1,CAPVAL2,CAPPED2,CAPVAL3,CAPPED3,CAPVAL4,CAPPED4,CAPVAL5,CAPPED5,CAPVAL6,CAPPED6
#         ,AvgSin,PROP_12,Num_Data

df_CLAG_MR_2022["CAPVAL1"]=df_CLAG_MR_2022.iloc[:,4]
df_CLAG_MR_2022["CAPPED1"]=0
df_CLAG_MR_2022["CAPVAL2"]=df_CLAG_MR_2022.iloc[:,4]
df_CLAG_MR_2022["CAPPED2"]=0
df_CLAG_MR_2022["CAPVAL3"]=df_CLAG_MR_2022.iloc[:,4]
df_CLAG_MR_2022["CAPPED3"]=0
df_CLAG_MR_2022["CAPVAL4"]=df_CLAG_MR_2022.iloc[:,4]
df_CLAG_MR_2022["CAPPED4"]=0
df_CLAG_MR_2022["CAPVAL5"]=df_CLAG_MR_2022.iloc[:,4]
df_CLAG_MR_2022["CAPPED5"]=0
df_CLAG_MR_2022["CAPVAL6"]=df_CLAG_MR_2022.iloc[:,4]
df_CLAG_MR_2022["CAPPED6"]=0

df_CLAG_MR_2022["AvgSin"]=0
df_CLAG_MR_2022["PROP_12"]=0
df_CLAG_MR_2022["Num_Data"]=0

df_CLAG_MR_2022.head(2)




,COUNT,mid_x,mid_y,mid_z,Ag_ppm_BESTEL,NBKRT,AG_SUBZONAS,HOLEID,CAPVAL1,CAPPED1,...,CAPPED3,CAPVAL4,CAPPED4,CAPVAL5,CAPPED5,CAPVAL6,CAPPED6,AvgSin,PROP_12,Num_Data
0,2336,4072.2950,4850.2450,4199.8669,25.146513,32,42,A019,25.146513,0,...,0,25.146513,0,25.146513,0,25.146513,0,0,0,0
1,2337,4073.5668,4848.7522,4196.7885,40.000000,31,42,A019,40.000000,0,...,0,40.000000,0,40.000000,0,40.000000,0,0,0,0


In [12]:

breaker=0
iterador=0

#for zone in df_parametros.Valor[12][:1]:     ## borrar   [:2]
for zone in df_parametros.Valor[12]:     #################################  FIJO PARA PRUEBA  BORRAR
    
    #print("\n")
    print(" ") 
    print("zone:",zone)
    print(" ") 

    #  df_parametros.Valor[13]         ## LISTA DE AZIMUTH
    print("azimuth:",df_parametros.Valor[13][iterador])
    
    azimuth=df_parametros.Valor[13][iterador]
    cartesiana=0          ## <------ 

    if azimuth < 90:
        #print(1)
        cartesiana=90-azimuth 
        print(cartesiana)
    elif azimuth < 180:
        #print(2)
        cartesiana=360-(azimuth-90) 
        print(cartesiana)
    elif azimuth < 270:
        #print(3)
        cartesiana=270-(azimuth-180) 
        print(cartesiana)
    elif azimuth < 360:
        #print(4)
        cartesiana=180-(azimuth-270) 
        print(cartesiana)
        
    print("cartesiana:",cartesiana)
    rad=math.radians(cartesiana)    ## <------  APLICAR ACA SIN COS etc etc    
    print("radian:",rad)


    #df_parametros.Valor[14]                            ## LISTA DE ANISOTROPIA 
    print("ratio anisotropia:",df_parametros.Valor[14][iterador])
    ratio=df_parametros.Valor[14][iterador]
    

    ###for rock in df_parametros.Valor[8][:1]:## borrar   [3]
    for rock in df_parametros.Valor[8]:
        
        print(" ") 
        print("    zone:",zone)
        print("    rock:",rock)
        print(" ") 
        print(" ") 
        # $$ ITERAR COMPOSITOS:   ROCK ZONE FIXED
        print("    parametros de la Zona ",zone)
        print(" ")
        print("    parametros de la Roca ",rock)






        # $$ POLIGONO x COMPÓSITO   x,y
        x_SI= x - (df_parametros.Valor[3][1] / 2)
        y_SI= y + ((df_parametros.Valor[3][2]*ratio) / 2)
        
        x_II= x - (df_parametros.Valor[3][1] / 2)
        y_II= y - ((df_parametros.Valor[3][2]*ratio) / 2)
        
        x_SD= x + (df_parametros.Valor[3][1] / 2)
        y_SD= y + ((df_parametros.Valor[3][2]*ratio) / 2)
        
        x_ID= x + (df_parametros.Valor[3][1] / 2)
        y_ID= y - ((df_parametros.Valor[3][2]*ratio) / 2)



        for obs in df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T:  ## ITERAR INDICES
            
            print("        index:",obs)
            
            # $$ PRINT COORDENADAS COMPOSITO
            
            x=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].iloc[:,1][obs]
            y=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].iloc[:,2][obs]
            z=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].iloc[:,3][obs]
             
            print("        x:",x)
            print("        y:",y)
            print("        z:",z)  
            print(" ")        
            


    print(" ")
    print(" ")




 
zone: 41
 
azimuth: 140
310
cartesiana: 310
radian: 5.410520681182422
ratio anisotropia: 2.5
 
    zone: 41
    rock: 31
 
 
        index: 73
        x: 3185.3567
        y: 4329.8712
        z: 4116.0064
 
        index: 74
        x: 3187.2542
        y: 4327.8264
        z: 4113.6524
 
        index: 75
        x: 3188.8393
        y: 4326.1182
        z: 4111.6847
 
        index: 76
        x: 3190.3981
        y: 4324.4384
        z: 4109.7487
 
        index: 77
        x: 3191.7748
        y: 4322.9548
        z: 4108.0381
 
        index: 116
        x: 3822.7913
        y: 4814.4253
        z: 4199.8413
 
        index: 117
        x: 3833.4343
        y: 4803.2334
        z: 4177.6951
 
        index: 118
        x: 3834.6317
        y: 4801.9742
        z: 4175.2496
 
        index: 119
        x: 3835.8321
        y: 4800.7119
        z: 4172.8071
 
        index: 120
        x: 3837.0355
        y: 4799.4465
        z: 4170.3678
 
        index: 121
        x: 3838.241